<a href="https://colab.research.google.com/github/Pieter-devries/python_playground/blob/main/public_query_favorite_contents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run me to initialize SDK
Open cell to check individual functions

In [ ]:
!pip install looker_sdk
!pip install pandas

In [ ]:
import looker_sdk #Note that the pip install required a hyphen but the import is an underscore.
from looker_sdk import models as mdls #This is required for high-level sdk calls
import os #We import os here in order to manage environment variables for the tutorial. You don't need to do this on a local system or anywhere you can more conveniently set environment variables.

import json #This is a handy library for doing JSON work.

from pandas import json_normalize


In [ ]:
# Setup Environment Settings
# Setup Instance/SDK information
os.environ["LOOKERSDK_BASE_URL"] = "https://xxx.cloud.looker.com" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

# Setup User login information
# Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.
print("All environment variables set.")

All environment variables set.


In [ ]:
# Initialize sdk
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

Looker SDK 4.0 initialized successfully.


# Create User/Dashboard list and functions

In [ ]:
# Create User list with email/user_id values
user_list = dict()
users = sdk.all_users()
for user in users:
  if user['email'] and not user['is_disabled']: #removing users that don't have emails setup and aren't disabled
    user_list[user['email']] = user['id']
print(user_list)

In [ ]:
# Create Dashboard list with title/content_metadata_id values
dashboard_list = dict()
dashboards = sdk.all_dashboards()
for dashboard in dashboards:
  if dashboard['content_metadata_id']: #removing dashboards without content_metadata_id
    dashboard_list[dashboard['title']] = dashboard['content_metadata_id']
print(dashboard_list)

In [ ]:
# Check all dashboards if the current user has favorited them or not
favorited_content = dict()

def retrieve_favorite_content(): #Create a function so we can loop it for each user
  for k,v in dashboard_list.items():
    response = sdk.search_content_favorites(content_metadata_id=v)
    if response:
      email = [k for k,v in user_list.items() if v == response[0]['user_id']][0] #Change user_id into email for data readability
      dashboard = [k for k,v in dashboard_list.items() if v == response[0]['content_metadata_id']][0] #Change content_metadata_id into dashboard name for data readability
      if email in favorited_content: #If we already have a favorited content, we need to append the new content
        favorited_content[email].append(dashboard)
      else: #If we don't have any favorited content yet, we add the new content as a list
        favorited_content[email] = []
        favorited_content[email].append(dashboard)



#Retrieve favorited content for all users

In [ ]:
for k,v in user_list.items():
  sdk.auth.logout() #This is required if you stopped the sdk mid-way
  sudo_auth = sdk.login_user(v)
  sdk.auth.login_user(v)
  retrieve_favorite_content()
print(favorited_content)